In [0]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dropout,AveragePooling2D, merge, Activation,SeparableConv2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 100

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [49]:
input = Input(shape=(img_height, img_width, channel,))

# Layer 1
x = SeparableConv2D(32, (5,5),border_mode='same', name='Sep_conv_1')(input)
x = Activation('relu')(x)
x = BatchNormalization(name='norm_1')(x)

layer1 = x

# Layer 2
x = Conv2D(64, (5,5), name='Conv_1',border_mode='same')(x)
x = Activation('relu')(x)
x = BatchNormalization(name='norm_2')(x)

layer2 = x

# Layer 3
x = Conv2D(128, (5,5), name='Conv_2', border_mode='same')(x)
x = Activation('relu')(x)
x = BatchNormalization(name='norm_3')(x)

layer3 = x

skip1 = concatenate([layer1, layer3])

# Layer 4
x = SeparableConv2D(256, (5,5), name='Sep_conv_2',border_mode='same')(skip1)
x = Activation('relu')(x)
x = BatchNormalization(name='norm_4')(x)

layer4 = x

# Layer 5
skip2 = concatenate([layer1, layer4])

x = MaxPooling2D(pool_size=(2, 2))(skip2)


# Layer 6
x = SeparableConv2D(32, (3,3), border_mode='same', name='Sep_conv_3', use_bias=False)(x)
x = Activation('relu')(x)
x = BatchNormalization(name='norm_6')(x)

layer6 = x

# Layer 7
skip3 = concatenate([layer1, layer4])

#layer61 = keras.layers.UpSampling3D(size=(2, 2, 1), data_format=None)(layer6)
skip31 = AveragePooling2D(2)(skip3)

x = concatenate([skip31, layer6])
x = Conv2D(64, (5,5), name='Conv_3', border_mode='same')(x)
x = Activation('relu')(x)
x = BatchNormalization(name='norm_7')(x)

layer7 = x
print(layer7.shape)

# Layer 8
skip4 = concatenate([layer3, layer4])
skip41 = AveragePooling2D(2)(skip4)

skip5 = concatenate([layer6, layer7])

x = concatenate([skip41,skip5])

x = SeparableConv2D(128, (3,3), border_mode='same', name='Sep_conv_4', use_bias=False)(x)
x = Activation('relu')(x)
x = BatchNormalization(name='norm_8')(x)

layer8 = x

# Layer 9
skip6 = concatenate([layer1,layer3, layer4])
skip61 = AveragePooling2D(2)(skip6)

skip7 = concatenate([layer6,layer7, layer8])

x = concatenate([skip61,skip7])

x = SeparableConv2D(256, (5,5), border_mode='same', name='Sep_conv_5', use_bias=False)(x)
x = Activation('relu')(x)
x = BatchNormalization(name='norm_9')(x)

layer9 = x

# Layer 10
skip8 = concatenate([layer1, layer4])
skip81 = AveragePooling2D(2)(skip8)

skip9 = concatenate([layer6,layer8, layer9])

x = concatenate([skip81,skip9])

x = MaxPooling2D(pool_size=(2, 2))(x)

# Layer 11
skip10 = AveragePooling2D(2)(layer7)
x = concatenate([skip10,x])
x = Conv2D(32, (5,5), name='Conv_4',border_mode='same')(x)
x = Activation('relu')(x)
x = BatchNormalization(name='norm_11')(x)

layer11 = x

# Layer 12
skip11 = concatenate([layer1, layer2])
skip111 = AveragePooling2D(4)(skip11)

skip12 = AveragePooling2D(2)(layer8)

x = concatenate([skip111,skip12,x])

x = SeparableConv2D(64, (5,5), border_mode='same', name='Sep_conv_6', use_bias=False)(x)
x = Activation('relu')(x)
x = BatchNormalization(name='norm_12')(x)

layer12 = x

# Layer 13
skip13 = concatenate([layer2, layer3])
skip131 = AveragePooling2D(4)(skip13)

skip14 = AveragePooling2D(2)(layer6)

x = concatenate([layer12,layer11,skip131,skip14])

x = Conv2D(128, (3,3), name='Conv_5',border_mode='same',)(x)
x = Activation('relu')(x)
x = BatchNormalization(name='norm_13')(x)

layer13 = x

# Layer 14

skip15 = concatenate([layer12, layer13])

skip16 = concatenate([layer6, layer8])
skip161 = AveragePooling2D(2)(skip16)

skip17 = concatenate([layer1, layer3,layer4])
skip171 = AveragePooling2D(4)(skip17)

x = concatenate([skip15,skip161,skip171])

x = SeparableConv2D(256, (5,5), border_mode='same', name='Sep_conv_7', use_bias=False)(x)
x = Activation('relu')(x)
x = BatchNormalization(name='norm_14')(x)

layer14 = x

# Layer 15

skip18 = concatenate([layer14, layer12])
skip19 = AveragePooling2D(4)(layer4)
skip20 = AveragePooling2D(2)(layer8)

x = concatenate([skip18,skip19,skip20])

x = Conv2D(256, 3, 3,)(x)
x = Conv2D(256, 3, 3,)(x)
x = Conv2D(256, 3, 3,)(x)
x = Conv2D(10, 2)(x)

x = Flatten()(x)
output = Activation('softmax')(x)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(32, (5, 5), name="Sep_conv_1", padding="same")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), name="Conv_1", padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), name="Conv_2", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(256, (5, 5), name="Sep_conv_2", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(32,

(?, 16, 16, 64)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), name="Conv_4", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:115: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(64, (5, 5), name="Sep_conv_6", use_bias=False, padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:129: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), name="Conv_5", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:147: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(256, (5, 5), name="Sep_conv_7", use_bias=False, padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:161: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:162: U

In [50]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
Sep_conv_1 (SeparableConv2D)    (None, 32, 32, 32)   203         input_33[0][0]                   
__________________________________________________________________________________________________
activation_267 (Activation)     (None, 32, 32, 32)   0           Sep_conv_1[0][0]                 
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 32, 32, 32)   128         activation_267[0][0]             
__________________________________________________________________________________________________
Conv_1 (Co

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [52]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

W0619 05:59:08.303069 140551772518272 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 110s 2ms/step - loss: 1.5691 - acc: 0.5227 - val_loss: 1.1944 - val_acc: 0.5852
Epoch 2/100
50000/50000 [==============================] - 90s 2ms/step - loss: 0.9713 - acc: 0.6627 - val_loss: 0.9644 - val_acc: 0.6673
Epoch 3/100
50000/50000 [==============================] - 89s 2ms/step - loss: 0.8063 - acc: 0.7220 - val_loss: 0.9895 - val_acc: 0.6653
Epoch 4/100
50000/50000 [==============================] - 89s 2ms/step - loss: 0.6883 - acc: 0.7633 - val_loss: 0.9412 - val_acc: 0.7117
Epoch 5/100
50000/50000 [==============================] - 89s 2ms/step - loss: 0.6127 - acc: 0.7905 - val_loss: 1.1164 - val_acc: 0.6560
Epoch 6/100
50000/50000 [==============================] - 89s 2ms/step - loss: 0.5397 - acc: 0.8159 - val_loss: 0.9710 - val_acc: 0.7199
Epoch 7/100
50000/50000 [==============================] - 89s 2ms/step - loss: 0.5005 - acc: 0.8318 - val_loss: 1.1705 -

In [53]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Assignment7.h5")
print("Saved the model to disk")

10000/10000 [==============================] - 7s 706us/step
Test loss: 14.283856028747559
Test accuracy: 0.1138
Saved the model to disk


In [0]:
model.save_weights("Assignment7.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Assignment7.h5')